In [1]:
from gurobipy import *
from operator import le
from unicodedata import name
from numpy import conjugate
import random
from typing import List
import pandas as pd

In [2]:
def discreteTime(timeWindow, intervalSize):   # O(lenghtOftimeWindow/size)
    tw = []
    currtime = timeWindow[0]
    while (currtime <= timeWindow[1]):
        tw.append(currtime)
        currtime = currtime+intervalSize
    return tw

def allPathsSourceTarget(graph: List[List[int]]) -> List[List[int]]:
    stack = [[0]]
    paths = []
    maxrun = 10000
    counter = 0
#     print("paths",paths)
#     print("just in")
    while stack:
        path = stack.pop()
        node = path[-1]
#         print("Node under consideration :",node)
        if node == len(graph) - 1:
            paths.append(path)
#             print("WOHOOOO",paths)
            
            counter+=1
#             print("found a path",counter," : ",path)
            if(counter == maxrun):
                return paths
            continue
            
        for new_node in graph[node]:
#             print("Nodes coming out :",new_node)
            stack.append((path + [new_node]))
                    
#     print(paths)    
    return paths

def distinctCustomer(path,nodeCount):
    customer = 0
    visited = [] # stores index of visited customer
    for i in range(1,len(path)-1):
        node = path[i]
        for j in range(0,len(nodeCount)):
            if(node in nodeCount[j]):
                if(j not in visited):
                    visited.append(j)
                    customer+=1
    return customer,visited
def getmin(path,ans,new_edges):
    for i in range(0,len(path)-1):
        edge = (path[i],path[i+1])
        minVal = 99 # any number > 1
        for key in new_edges:
            if(new_edges[key] == edge):
                minVal = min(minVal,ans[key])
    return minVal

def maxCustomerpath(paths,nodeCount):
    maxPaths = 0
    maxCustomer = 0
    for i in range(0,len(paths)):
        path = paths[i]
        customer,visited = distinctCustomer(path,nodeCount)
        if(maxCustomer < customer):
            maxCustomer = customer
            maxPaths = visited # customers visited
            pathdirection = path
            
    return maxCustomer,maxPaths,pathdirection
        

In [3]:


# # 20 customer -> 1.1 minute
# # 30 customers -> 5 minute    1.69 minutes
# # 40                          2.45 min
# customers = 100
# timeWindows = []
# for i in range(0,customers):
#     st = random.randint(1,2)*10
#     et = random.randint(1,3)*10 + st
#     tw = [st,et]
#     timeWindows.append(tw)

# print("timeWindows =", timeWindows)
# print()

# discreteTimeWindows = []
# for i in timeWindows:
#     discreteTimeWindows.append(discreteTime(i, 10))

# # print("discreteTimeWindows ", discreteTimeWindows)
# # print()



# timeMatrix = []
# for i in range(0,customers):
#     timeMatrix.append([0]*customers)
# for i in range(0,customers):
#     for j in range(0,customers):
#         if(i==j):
#             timeMatrix[i][j] = 0
#         elif (i<j):
#             timeMatrix[i][j] = random.randint(1,10)*10
#             timeMatrix[j][i] = timeMatrix[i][j]

# print("timeMatrix = ", timeMatrix)
# print()



In [4]:
customers = 70
df = pd.read_csv("data_fin.csv")
start = 0
speed = 0.5 # 30km/hr
df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
df.reset_index(inplace = True)


#remove unreachable if(starting time + travelling > end )


print(len(df))
# df = df[ df['DUE DATE'] >= start + (df['XCOORD.']+df['YCOORD.'])/speed ]
timeWindows = [] 
for i in range(0,customers):
    st = df['READY TIME'][i]-(df['READY TIME'][i]%10)
    et = df['DUE DATE'][i]-(df['DUE DATE'][i]%10)
    tw = [st,et]
    timeWindows.append(tw)

discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i,10))



timeMatrix = []
speed = 0.5 # 30km/hr
for i in range(0,customers):
    timeMatrix.append([0]*customers)
for i in range(0,customers):
    for j in range(0,customers):
        if(i==j):
            timeMatrix[i][j] = 0
        elif (i<j):
            c= int((abs(df['XCOORD.'][i]-df['XCOORD.'][j])+abs(df['YCOORD.'][i]-df['YCOORD.'][j]))/speed)
            if(c%10 == 0 or c%10 == 5):
                timeMatrix[i][j] = c
                timeMatrix[j][i] = timeMatrix[i][j]
            elif(c%10 < 5):
                timeMatrix[i][j] = c-(c%10)
                timeMatrix[j][i] = timeMatrix[i][j]
            elif(c%10 > 5):
                timeMatrix[i][j] = c+(10 - c%10)
                timeMatrix[j][i] = timeMatrix[i][j]                

98


In [ ]:
VEHICLES = 0

while(len(timeWindows)!=0):
    m1 = Model('VRPTW')
    VEHICLES+=1
    count = 0
    nodeCount = []  # assigns a number to node using count and stores its value

    for i in range(0, len(discreteTimeWindows)):
        c = []
        for j in range(0, len(discreteTimeWindows[i])):
            c.append(count+1)
            count += 1
        nodeCount.append(c)

    print("Number of customers",len(nodeCount))
    print("Nodes ",nodeCount)



    edgeFlows = {}
    edgeNumber = 0
    x = [] 
    edges = {}
    edgesList = []  # stores edge number going out of ith node
    edgesList.append([])  # for s
    
    
    
    # GRAPH CONSTRUCTION
    # out of s to starting of each customer
    e = []
    for i in range(0, len(nodeCount)):
    #     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
    #     m1.update()
        edges['x[%i]' % edgeNumber] = (0, nodeCount[i][0])
        edgeFlows[edgeNumber] = (0, nodeCount[i][0])
        e.append(edgeNumber)
        edgeNumber += 1
    edgesList[0] = e

    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])-1):
            e = []
    #         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
    #         m1.update()
            edges['x[%i]' % edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
            edgeFlows[edgeNumber] = (nodeCount[i][j], nodeCount[i][j+1])
            # add edge number in edge list
            e.append(edgeNumber)
            edgesList.append(e)
            edgeNumber += 1
        edgesList.append([])
#     print("Edges List:", edgesList)


    for i in range(len(discreteTimeWindows)):
        tu = nodeCount[i][-1]
        start_utime = discreteTimeWindows[i][0]
        end_utime = discreteTimeWindows[i][-1]

        # stores edges reachable from u
        for k in range(0, len(discreteTimeWindows)):
            sv = nodeCount[k][0]

            start_vtime = discreteTimeWindows[k][0]
            end_vtime = discreteTimeWindows[k][-1]


            if(i != k and start_utime + timeMatrix[i][k] > end_vtime):
#                 print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
                print("",end="")
                  # not reachable

            elif(i != k and start_vtime > end_utime + timeMatrix[i][k]):
                # edge bw (u,tu) -> (v,sv)
#                 print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
                e = []
    #             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))

                edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
                edgeFlows[edgeNumber] = (tu, sv)
#                 print(tu, "->", sv)
                e.append(edgeNumber)
                edgesList[tu].extend(e)
                edgeNumber += 1


            elif(i != k and start_vtime == end_utime + timeMatrix[i][k]):
#                 print("customer",i,"to",k,"start_utime ",start_utime," end_utime ",end_utime," start_vtime",start_vtime," end_vtime",end_vtime)
                e = []
    #             x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))


                edges['x[%i]' % edgeNumber] = (tu, sv)  # u -> v reachable
#                 print(tu, "->", sv)
                edgeFlows[edgeNumber] = (tu, sv)
                e.append(edgeNumber)
                edgesList[tu].extend(e)
                edgeNumber += 1


            elif (i != k and start_vtime < end_utime + timeMatrix[i][k]):

                firstReachableNode_index = 0  # first such that node >= sv
                for j in range(0, len(discreteTimeWindows[i])):
                    if(timeMatrix[i][k] + discreteTimeWindows[i][j] >= start_vtime):
                        firstReachableNode_index = j
                        break
                # now starting from firstReachableNode make edges until feasible
                for l in range(firstReachableNode_index, len(discreteTimeWindows[i])):
                    e = []
                    reachingTime = discreteTimeWindows[i][l]+timeMatrix[i][k]
                    for r in range(0, len(discreteTimeWindows[k])):
                        if(discreteTimeWindows[k][r] >= reachingTime):
#                             print(edgesList[nodeCount[i][l]])
#                             print(nodeCount[i][l], "->", nodeCount[k][r])
                            # edge bw these two
    #                         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
                            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))

                            edges['x[%i]' % edgeNumber] =(nodeCount[i][l], nodeCount[k][r])  # u -> v reachable
                            edgeFlows[edgeNumber] = (nodeCount[i][l], nodeCount[k][r])

                            e.append(edgeNumber)
                            edgesList[nodeCount[i][l]].extend(e)
#                             print("hdhdh", edgesList[nodeCount[i][l]])
                            edgeNumber += 1
                            break




#     print('edge list b4 s t', edgesList)
#     print()

    # into t (except s)
    e = []
    for i in range(0, len(nodeCount)):
    #     x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
        x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))
        edges['x[%i]' % edgeNumber] = (nodeCount[i][-1], -1)
        edgeFlows[edgeNumber] = (nodeCount[i][-1], -1)
        # add edge number in edge list
        edgesList[nodeCount[i][-1]].append(edgeNumber)
        edgeNumber += 1
    edgesList.append([])


#     print("Node Count after s and t:", nodeCount)
#     print()

    # GRAPH Construction OVER
    # if I add it to  0 edge i'll get flow for 1 at the same edge  , add to 1 customer 2 customer


    offset = edgeNumber

#     print('edge list', edgesList)
#     print()
#     print("edges", edges)
#     print()
#     print('edge number', edgeNumber)
#     print()

    # print("edge flows:", edgeFlows)

    flows = [0]*customers
    for i in range(customers):
        flows[i] = edgeFlows.copy()
        keys = edgeFlows.keys()
        for j in keys:  # stores key values of edgeflows i.e 0 1 2
            # all x[] > offset are for flow variables
    #         x[edgeNumber] = solver.NumVar(0, 1, 'x[%i]' % edgeNumber)
            x.append(m1.addVar( lb=0, ub=1 , vtype= GRB.CONTINUOUS, name='x[%i]' % edgeNumber ))

            # remove key and replace it by edge
            t = flows[i].pop(j)
            flows[i][edgeNumber] = t
            edgeNumber += 1

    # print("flows", flows)

    noc = 0
    # # contraint for outgoing flow for a customer
    for i in range(0, len(nodeCount)):  # choose one list from nodeCount
    #     contraint = [0] * (edgeNumber)
        coeff = [] # store where coeff = 1
        for j in range(0, len(nodeCount[i])):  # se
            node = nodeCount[i][j]
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                if(v not in nodeCount[i]):
                        # print('hello')
                        # print("u", u, "v", v)
                        coeff.append(x[e])

        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1



    incomingEdgeList = []
    for i in range(0, len(edgesList)):
        incomingEdgeList.append([])  # adding lists in a list
    for i in edges:
        v = edges[i][1]  # u -> v

        edge_no = int(i[2:-1])
        incomingEdgeList[v].append(edge_no)

#     print("incoming edge list: ", incomingEdgeList)
#     print()

    #  constraint for incoming for a customer
    for i in range(0, len(nodeCount)):
        coeff = []
        contraint = [0] * (edgeNumber)  # indicates number of edges
        for j in range(0, len(nodeCount[i])):
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                u = edges['x[%i]' % e][0]
                v = edges['x[%i]' % e][1]
                    # print("u", u, "v", v)
                if(u not in nodeCount[i]):
                        # print('hello')
                        # print("u", u, "v", v)
                        coeff.append(x[e])
        m1.addConstr(( sum(coeff) == 1  ))
        noc+=1

    # constraint for a node incoming = outgoing
    for i in range(0, len(nodeCount)):
        for j in range(0, len(nodeCount[i])):
            coeff = []
            contraint = [0] * (edgeNumber)
            node = nodeCount[i][j]
            for k in range(0, len(incomingEdgeList[node])):
                e = incomingEdgeList[node][k]
                contraint[e] = 1
                coeff.append(x[e])
            for k in range(0, len(edgesList[node])):
                e = edgesList[node][k]
                contraint[e] = -1
                coeff.append(-x[e])
            m1.addConstr(( sum(coeff) == 0 ))
            noc+=1  
            

               
    # #  0<= xij - fij    f<x => 0<=x-f
    for j in range(len(flows)):
        diff = offset*j+offset
        for k in flows[j].keys():
            coeff = [] 
            coeff.append(-x[k])
            coeff.append(x[k-diff])
            m1.addConstr(( sum(coeff) >= 0 ))
            

    
    ## print(contraint," >= 0")

    # constraint for a particular customer outgoing == 0 for c the customer for rest dont care
    for c in range(customers):
        addn = offset*(c+1)
        for i in range(0, len(nodeCount)):
            
            if(i == c):
                
            # indicates number of edges
                coeff = []
                for j in range(0, len(nodeCount[i])):
                    
                    node = nodeCount[i][j]
                    for k in range(0, len(edgesList[node])):
                        
                        e = edgesList[node][k]
                        u = edges['x[%i]' % e][0]
                        v = edges['x[%i]' % e][1]
                        # print("u", u, "v", v)
                        if(v not in nodeCount[i]):
                            
                            # print('hello')
                            # print("u", u, "v", v)
                            edge = 'x[%i]' % e
                            node = edges[edge][0]
        #                     print("node:",node)
                            coeff.append(x[e+addn])
                m1.addConstr(( sum(coeff) == 0 ))


#     print('done w edges')        






#     print("CONSTRAINT for incoming = 1 ")
    for c in range(customers):
        
        addn = offset*(c+1)

        for i in range(0, len(nodeCount)):
            
            # indicates number of edges
            coeff = []
            if( i == c):
                for j in range(0, len(nodeCount[i])):
                    
                    node = nodeCount[i][j]
                    for k in range(0, len(incomingEdgeList[node])):
                        
                        e = incomingEdgeList[node][k]
                        u = edges['x[%i]' % e][0]
                        v = edges['x[%i]' % e][1]
                        # print("u", u, "v", v)
                        if(u not in nodeCount[i]):
                            
                            # print('hello')
                            # print("u", u, "v", v)
                            coeff.append(x[e+addn])
                m1.addConstr(( sum(coeff) == 1 ))




    # # flow conservation  for every node except those present in customer = c

    for c in range(customers):
        addn = offset*c+offset
        for i in range(0, len(nodeCount)):
            if ( i != c):
                for j in range(0, len(nodeCount[i])):
                    coeff = []
                    node = nodeCount[i][j]
                    for k in range(0, len(incomingEdgeList[node])):
                        e = incomingEdgeList[node][k]
                        coeff.append(x[e+addn])
                    for k in range(0, len(edgesList[node])):
                        e = edgesList[node][k]
                        coeff.append(-x[e+addn])
                    m1.addConstr(( sum(coeff) == 0 ))
                                                       
    
                                 
                                 
                                 
    objectivefun = []  
    for i in range(0, len(edgesList[0])):
                                 objectivefun.append(x[edgesList[0][i]])
    obj = sum(objectivefun)
    m1.setObjective(obj,GRB.MINIMIZE)


    m1.optimize()
    print("Runtime is :",m1.Runtime)
    print("objective value : ",m1.objVal)
                                 
    ans = {}
    for v in m1.getVars()[:offset]:
        if (v.x > 0):
            ans[(v.varName[2:-1])]=v.x

                                 
    new_edges = {}   # stores only those edges which are required
    for key in edges:
        if((key[2:-1]) in ans):
            new_edges[(key[2:-1])] = edges[key]
                                 
                                 
    graphForpaths = []
    for i in range(len(edgesList)): # edgesList contains outgoing from each node
        l = []
        for key in new_edges:
            if(new_edges[key][0] == i):
                if(new_edges[key][1] == -1):
                    l.append(len(edgesList)-1)
                else:
                    l.append(new_edges[key][1])
        graphForpaths.append(l)
      
    
    paths = allPathsSourceTarget(graphForpaths)
    for i in range(len(paths)):
        paths[i][-1] = -1
                              
    distict_customers,customer_nodes,path =  maxCustomerpath(paths,nodeCount)  
    customer_nodes.sort(reverse = True)
    
                                 
    #modify time-windows
    print("customer nodes",customer_nodes)
    for i in range(0,len(customer_nodes)):
        del timeWindows[customer_nodes[i]]
                                 
    discreteTimeWindows = []
    for i in timeWindows:
        discreteTimeWindows.append(discreteTime(i, 10))
                                 

    for row in customer_nodes:
        del timeMatrix[row]

    for row in range(0,len(timeMatrix)):                         
        for col in customer_nodes:
            del timeMatrix[row][col]
    
    print("----------------------------------------------------------------------------------------------------")
    print("Removing ",distict_customers," distinct customer",customer_nodes,"and path is ",path)
    print("----------------------------------------------------------------------------------------------------")
    
                                 

Set parameter Username
Academic license - for non-commercial use only - expires 2023-06-11
Number of customers 70
Nodes  [[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101], [102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116], [117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131], [132, 133, 134, 135, 136, 137, 138], [139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152], [153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170], [171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181], [182, 183, 184, 185, 186, 187, 188, 189, 190], [191, 192, 193, 194,

In [ ]:
print("Vehicles required : ",VEHICLES)

In [ ]:
print(customers)

In [ ]:
for v in m1.getVars():
    print('%s: %g' % (v.varName,v.x))

In [ ]:
# print("Number of Constraints :",noc)
# print("Runtime is :",m1.Runtime)

In [ ]:
ans = {}
for v in m1.getVars()[:offset]:
    if (v.x > 0):
#         print(v.varName[2:-1])
        ans[(v.varName[2:-1])]=v.x
ans

In [ ]:
edges
new_edges = {}   # stores only those edges which are required
for key in edges:
    if((key[2:-1]) in ans):
        new_edges[(key[2:-1])] = edges[key]
new_edges

In [ ]:
graphForpaths = []
for i in range(len(edgesList)): # edgesList contains outgoing from each node
    l = []
    for key in new_edges:
        if(new_edges[key][0] == i):
            if(new_edges[key][1] == -1):
                l.append(len(edgesList)-1)
            else:
                l.append(new_edges[key][1])
    graphForpaths.append(l)
graphForpaths    

In [ ]:
paths = allPathsSourceTarget(graphForpaths)
for i in range(len(paths)):
    paths[i][-1] = -1
paths

In [ ]:
offset

In [ ]:
nodeCount

In [ ]:
def distinctCustomer(path,nodeCount):
    customer = 0
    visited = [] # stores index of visited customer
    for i in range(1,len(path)-1):
        node = path[i]
        for j in range(0,len(nodeCount)):
            if(node in nodeCount[j]):
                if(j not in visited):
                    visited.append(j)
                    customer+=1
    return customer,visited

In [ ]:
def maxCustomerpath(paths,nodeCount):
    maxPaths = 0
    maxCustomer = 0
    for i in range(0,len(paths)):
        path = paths[i]
        customer,visited = distinctCustomer(path,nodeCount)
        if(maxCustomer < customer):
            maxCustomer = customer
            maxPaths = visited
            
    return maxCustomer,maxPaths
        
        

distict_customers,customer_nodes =  maxCustomerpath(paths,nodeCount)      
distict_customers

In [ ]:
customer_nodes.sort(reverse = True)
customer_nodes

In [ ]:
timeWindows = [[0, 30], [1, 30], [2, 20], [3, 40], [4, 40], [5, 50], [6, 30], [7, 30], [8, 30], [9, 30]]
# discreteTimeWindows [[20, 30], [20, 30], [10, 20], [20, 30, 40], [20, 30, 40], [20, 30, 40, 50], [10, 20, 30], [10, 20, 30], [20, 30], [20, 30]]
# timeMatrix =  [[0, 40, 70, 20, 90, 90, 60, 80, 40, 30], [40, 0, 10, 40, 40, 70, 50, 30, 30, 60], [70, 10, 0, 80, 100, 60, 50, 50, 70, 100], [20, 40, 80, 0, 80, 70, 40, 10, 70, 80], [90, 40, 100, 80, 0, 100, 10, 100, 40, 40], [90, 70, 60, 70, 100, 0, 10, 20, 20, 90], [60, 50, 50, 40, 10, 10, 0, 40, 80, 10], [80, 30, 50, 10, 100, 20, 40, 0, 100, 80], [40, 30, 70, 70, 40, 20, 80, 100, 0, 60], [30, 60, 100, 80, 40, 90, 10, 80, 60, 0]]

In [ ]:
#modify time-windows
for i in range(0,len(customer_nodes)):
    del timeWindows[customer_nodes[i]]
timeWindows


In [ ]:
discreteTimeWindows = []
for i in timeWindows:
    discreteTimeWindows.append(discreteTime(i, 10))
discreteTimeWindows

In [ ]:
timeMatrix =  [[0, 1, 2, 3, 4,5, 6, 7, 8, 9], [40, 0, 10, 40, 40, 70, 50, 30, 30, 60], [70, 10, 0, 80, 100, 60, 50, 50, 70, 100], [20, 40, 80, 0, 80, 70, 40, 10, 70, 80], [90, 40, 100, 80, 0, 100, 10, 100, 40, 40], [90, 70, 60, 70, 100, 0, 10, 20, 20, 90], [60, 50, 50, 40, 10, 10, 0, 40, 80, 10], [80, 30, 50, 10, 100, 20, 40, 0, 100, 80], [40, 30, 70, 70, 40, 20, 80, 100, 0, 60], [30, 60, 100, 80, 40, 90, 10, 80, 60, 0]]

In [ ]:
#modify time-Matrix
# timeMatrix = [[1,2,3],[2,1,3],[3,2,1]]
# l=[2]
for row in customer_nodes:
    del timeMatrix[row]
    
for row in range(0,len(timeMatrix)):
    for col in customer_nodes:
        del timeMatrix[row][col]
    
    
    
timeMatrix

In [ ]:
print(VEHICLES)